In [1]:
import cv2
import os

# --- Input video path ---
video_path = "fingering__expt_video (1).mp4"

# --- Output directory for frames ---
output_dir = "frames_5fps"
os.makedirs(output_dir, exist_ok=True)

# --- Open the video ---
cap = cv2.VideoCapture(video_path)

# Get original FPS of video
fps = cap.get(cv2.CAP_PROP_FPS)
print(f"Original FPS: {fps}")

# Define extraction rate (5 fps)
target_fps = 5
frame_interval = int(fps /target_fps)  # pick every Nth frame

frame_count = 0
saved_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Save every Nth frame
    if frame_count % frame_interval == 0:
        frame_name = os.path.join(output_dir, f"frame_{saved_count:04d}.jpg")
        cv2.imwrite(frame_name, frame)
        saved_count += 1

    frame_count += 1

cap.release()
print(f"✅ Done! Extracted {saved_count} frames at {target_fps} fps.")


Original FPS: 23.803304687336972
✅ Done! Extracted 207 frames at 5 fps.


In [2]:
import cv2
import os
import numpy as np

# --- Configuration ---
# Path to the folder containing the original frames
input_folder = 'frames_5fps'

# Path to the folder where processed frames will be saved
output_folder = 'output'

# --- ADJUSTED OUTPUT SIZE ---
# Increased the final dimensions to 512x512 for a larger, more detailed image.
IMG_WIDTH = 512
IMG_HEIGHT = 512

# --- Preprocessing Script ---

# 1. Create the output directory if it doesn't already exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"Created directory: {output_folder}")

# 2. Get a sorted list of all frame filenames from the input folder
try:
    files = sorted(os.listdir(input_folder))
except FileNotFoundError:
    print(f"Error: The input folder '{input_folder}' was not found.")
    print("Please make sure this script is in the same directory as your frames folder.")
    exit()

print(f"Starting LARGER preprocessing ({IMG_WIDTH}x{IMG_HEIGHT}) for {len(files)} frames...")

# 3. Loop through each file in the directory
for filename in files:
    # Ensure we're only processing image files
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        # Construct the full path to the input image
        img_path = os.path.join(input_folder, filename)
        
        # Read the image from the file in color
        img = cv2.imread(img_path)
        if img is None:
            print(f"Warning: Could not read image {filename}. Skipping.")
            continue

        # --- Preprocessing Steps ---

        # a) Crop the image to capture the full circular dish
        height, width, _ = img.shape
        center_x, center_y = width // 2, height // 2
        crop_size = min(height, width, 950) 
        
        start_x = max(center_x - crop_size // 2, 0)
        end_x = min(center_x + crop_size // 2, width)
        start_y = max(center_y - crop_size // 2, 0)
        end_y = min(center_y + crop_size // 2, height)
        
        cropped_img = img[start_y:end_y, start_x:end_x]

        # b) Apply Denoising on the color image
        denoised_img = cv2.medianBlur(cropped_img, 5)

        # c) Resize to the new, larger standard dimensions
        resized_img = cv2.resize(denoised_img, (IMG_WIDTH, IMG_HEIGHT))

        # d) Save the final processed color image to the output folder
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, resized_img)

print("\nLarge-format preprocessing complete!")
print(f"All processed color frames have been saved as {IMG_WIDTH}x{IMG_HEIGHT} images in the '{output_folder}' folder.")



Starting LARGER preprocessing (512x512) for 207 frames...

Large-format preprocessing complete!
All processed color frames have been saved as 512x512 images in the 'output' folder.


In [3]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import cv2
import numpy as np
import os

# Input and output directories
input_folder = "output"
mask_folder = "masks"
seg_folder = "segmented"

# Create folders if they don’t exist
os.makedirs(mask_folder, exist_ok=True)
os.makedirs(seg_folder, exist_ok=True)

for file_name in sorted(os.listdir(input_folder)):
    if file_name.endswith(".jpg") or file_name.endswith(".png"):
        file_path = os.path.join(input_folder, file_name)

        # Read image
        img = cv2.imread(file_path)

        # Convert to HSV
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

        # Define HSV range for reddish/pink fingers
        lower = np.array([130, 50, 50])   # adjust if needed
        upper = np.array([179, 255, 255]) # upper red/pink

        # Create mask
        mask = cv2.inRange(hsv, lower, upper)

        # Clean mask
        kernel = np.ones((3,3), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=2)
        mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, kernel, iterations=1)

        # Apply mask
        segmented = cv2.bitwise_and(img, img, mask=mask)

        # Save mask (binary image)
        mask_path = os.path.join(mask_folder, f"mask_{file_name}")
        cv2.imwrite(mask_path, mask)

        # Save segmented image
        seg_path = os.path.join(seg_folder, f"seg_{file_name}")
        cv2.imwrite(seg_path, segmented)

print("✅ All masks saved in 'masks/' and segmented frames in 'segmented/'")


✅ All masks saved in 'masks/' and segmented frames in 'segmented/'
